In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.losses import Huber
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'F:\LSTM\Results\LSTM_3_Years.csv')

In [3]:
# Preprocess the data
data = df[['Precipitation', 'Temperature', 'SurfacePressure', 'RelativeHumidity','ZTD', 'Reflectivity','UComponentofWind']]  # Use relevant features
data = data.dropna()  #drops any row with at least one NaN value
data=data.rolling(5).mean()
data = data.dropna()
data.isnull().sum()

Precipitation       0
Temperature         0
SurfacePressure     0
RelativeHumidity    0
ZTD                 0
Reflectivity        0
UComponentofWind    0
dtype: int64

In [50]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Create sequences
def create_sequences(data, seq_length, output_length=6):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length - output_length + 1):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length:i + seq_length + output_length, 0])  # Assuming 'precipitation' is the first column
    return np.array(sequences), np.array(labels)

seq_length = 12  # Number of time steps to look back
output_length = 1  # Number of time steps to predict
X, y = create_sequences(data_scaled, seq_length, output_length)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (21030, 12, 7)
y_train shape: (21030, 1, 1)
X_test shape: (5258, 12, 7)
y_test shape: (5258, 1, 1)


In [51]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense
from keras.losses import Huber
# 1. Encoder-Decoder Setup

# Encoder
encoder_inputs = Input(shape=(seq_length, X_train.shape[2]))
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Create a model that outputs both the encoder's normal output and its states
encoder_model_with_context = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Decoder
decoder_inputs = Input(shape=(None, y_train.shape[2]))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(y_train.shape[2], activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

# Combined model for training
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss=Huber())


print("y_train shape:", y_train.shape)  # Expected: (num_samples, output_length)
print("X_train shape:", X_train.shape)  # Expected: (num_samples, seq_length, num_features)
print("encoder_outputs shape:", encoder_outputs.shape)  # Expected: (None, 128)
print("encoder_inputs shape:", encoder_inputs.shape)  # Expected: (None, seq_length, num_features)
print("decoder_inputs shape:", decoder_inputs.shape)  # Expected: (None, None, num_features)
print("decoder_outputs shape:", decoder_outputs.shape)  # Expected: (None, None, num_features)
print("state_h shape:", state_h.shape)  # Expected: (None, 128)
model.summary()

y_train shape: (21030, 1, 1)
X_train shape: (21030, 12, 7)
encoder_outputs shape: (None, 128)
encoder_inputs shape: (None, 12, 7)
decoder_inputs shape: (None, None, 1)
decoder_outputs shape: (None, None, 1)
state_h shape: (None, 128)


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)   │ (None, 12, 7)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_27 (InputLayer)   │ (None, None, 1)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_12 (LSTM)                │ [(None, 128), (None,      │          69,632 │ input_layer_26[0][0]       │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_13 (LSTM)                │ [(None, None, 128),       │          66,560 │ input_layer_27[0][0],      │
│                               │ (None, 128), (None, 128)] │                 │ lstm_12[0][1],             │
│                               │                           │                 │ lstm_12[0][2]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, None, 1)           │             129 │ lstm_13[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 136,321 (532.50 KB)

 Trainable params: 136,321 (532.50 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# 2. Prepare Training Data

# Since y_train has shape (num_samples, output_length), we need to create decoder_input_data with the same shape

# Ensure that decoder_input_data has the same number of time steps as y_train
decoder_input_data = np.zeros((y_train.shape[0], y_train.shape[1], y_train.shape[2]))  # Same shape as y_train but with all features from X_train
decoder_input_data[:, 1:, :] = y_train[:, :-1, :]
# Fill decoder_input_data with appropriate shifted values
# To match the number of time steps, we need to limit the time steps in X_train to match output_length
# for i in range(output_length):
#     decoder_input_data[:, i, :] = X_train[:, seq_length - output_length + i, :]  # Align based on the time step

# Reshape y_train to be 3D, with the same number of features as the model output
y_train_reshaped = np.expand_dims(y_train, -1)  # Now y_train has shape (num_samples, output_length, 1)

# To match the model's output shape, you can tile this across the number of features
# y_train_reshaped = np.tile(y_train_reshaped, (1, 1, X_train.shape[2]))  # Now y_train has shape (num_samples, output_length, num_features)

In [63]:
#######
import tensorflow as tf
from keras.callbacks import Callback

class ContextVectorCallback(Callback):
    def __init__(self, encoder_model):
        super().__init__()
        self.encoder_model = encoder_model

    def on_batch_end(self, batch, logs=None):
        # Fetching the input data from the batch directly
        if hasattr(self.model, 'data'):
            # If using a data generator
            batch_data = self.model.data[batch]
            print("this is part of if statement ",batch_data)
        else:
            # For in-memory data
            batch_data = self.model.input[0]  # Typically the first input is the encoder input
            print("this is part of else statement ",batch_data)
        
        if isinstance(batch_data, tf.Tensor):
            batch_inputs_eval = tf.keras.backend.eval(batch_data)
            print("this is part of if statement ",batch_inputs_eval)
        else:
            batch_inputs_eval = batch_data
            print("this is part of else statement ",batch_inputs_eval)

        # Debug: Print the shape and contents of batch_inputs_eval
        print(f"Batch {batch} - Input Shape: {batch_inputs_eval.shape}")
        
        # Predict the context vectors using the encoder
        encoder_output = self.encoder_model.predict(batch_inputs_eval)
        
        # Debug: Print the encoder output to understand its structure
        print(f"Batch {batch} - Encoder Output: {encoder_output}")
        
        # Unpack the encoder output if it's in the expected format
        if isinstance(encoder_output, list) and len(encoder_output) == 3:
            _, h, c = encoder_output
            # Print the context vectors (you can customize how you print them)
            print(f"Batch {batch} - Context Vector h: {h}, c: {c}")
        else:
            print(f"Unexpected encoder output: {encoder_output}")



In [64]:
#########
# Pass this callback when fitting the model
context_vector_callback = ContextVectorCallback(encoder_model_with_context)
model.fit([X_train, decoder_input_data], y_train,
          batch_size=32,
          epochs=20,
          validation_split=0.2,
          callbacks=[context_vector_callback])


Epoch 1/20
this is part of else statement  <KerasTensor shape=(None, 12, 7), dtype=float32, sparse=None, name=keras_tensor_96>
this is part of else statement  <KerasTensor shape=(None, 12, 7), dtype=float32, sparse=None, name=keras_tensor_96>
Batch 0 - Input Shape: (None, 12, 7)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
# 3. Train the Model
model.fit([X_train, decoder_input_data], y_train,
          batch_size=32,
          epochs=20,
          validation_split=0.2)

Epoch 1/20
526/526 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0081 - val_loss: 0.0013
Epoch 2/20
526/526 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0010 - val_loss: 4.3879e-04
Epoch 3/20
 59/526 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 3.7623e-04

In [34]:
#####model.save("E/D Training Model)

In [35]:
# 4. Define Inference Models

# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
decoder_model.summary()
encoder_model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)   │ (None, None, 1)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_20 (InputLayer)   │ (None, 128)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_21 (InputLayer)   │ (None, 128)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_7 (LSTM)                 │ [(None, None, 128),       │          66,560 │ input_layer_19[0][0],      │
│                               │ (None, 128), (None, 128)] │                 │ input_layer_20[0][0],      │
│                               │                           │                 │ input_layer_21[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, None, 1)           │             129 │ lstm_7[1][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,689 (260.50 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)          │ (None, 12, 7)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ [(None, 128), (None, 128),  │          69,632 │
│                                      │ (None, 128)]                │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,632 (272.00 KB)

 Trainable params: 69,632 (272.00 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# 5. Inference Function

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, y_train.shape[2]))

    # Populate the first token of target sequence with the start token.
    target_seq[0, 0, 0] = 1  # Start token, adjust according to your data

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length or find stop character.
        if (sampled_token_index == 0 or  # End token, adjust according to your data
                len(decoded_sentence) > output_length):
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, X_train.shape[2]))
        target_seq[0, 0, sampled_token_index] = 1

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# 6. Predict on a batch of new sequences
decoded_sentences = []
for i in range(len(X_test)):  # Iterate through all sequences in X_test
    input_seq = X_test[i:i+1]  # Select one sequence at a time
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences.append(decoded_sentence)
    print(f"Decoded sequence {i+1}:", decoded_sentence)  # Print each decoded sentence

# Optionally, print all decoded sentences after the loop
print("All Decoded sequences:", decoded_sentences)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Decoded sequence 1: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 2: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 3: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 4: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Decoded sequence 5: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Decoded sequence 6: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 7: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Decoded sequence 8: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 9: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Decoded sequence 10: [

In [ ]:
############
# 6. Predict on a batch of new sequences
decoded_sentences1 = []
for i in range(1,2500):  # Iterate through all sequences in X_test
    input_seq = X_test[i:i+1]  # Select one sequence at a time
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences1.append(decoded_sentence)
    print(f"Decoded sequence {i+1}:", decoded_sentence)  # Print each decoded sentence

# Optionally, print all decoded sentences after the loop
print("All Decoded sequences:", decoded_sentences)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Decoded sequence 2: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Decoded sequence 3: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 4: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Decoded sequence 5: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 6: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 7: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Decoded sequence 8: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Decoded sequence 9: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Decoded sequence 10: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Decoded sequence 11: [0

In [ ]:
############
# 6. Predict on a batch of new sequences
decoded_sentences2 = []
for i in range(2500,len(X_test)):  # Iterate through all sequences in X_test
    input_seq = X_test[i:i+1]  # Select one sequence at a time
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences2.append(decoded_sentence)
    print(f"Decoded sequence {i+1}:", decoded_sentence)  # Print each decoded sentence

# Optionally, print all decoded sentences after the loop
print("All Decoded sequences:", decoded_sentences)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Decoded sequence 2854: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Decoded sequence 2855: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Decoded sequence 2856: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Decoded sequence 2857: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Decoded sequence 2858: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Decoded sequence 2859: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Decoded sequence 2860: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Decoded sequence 2861: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Decoded sequence 2862: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Decoded sequence 2863: [0]
1/1 ━━━━

In [140]:
check=y_test.reshape(y_test.shape[0], 1)

ValueError: cannot reshape array of size 31542 into shape (5257,1)

In [139]:
check.shape

(31542, 1)

In [126]:
############  apply_nowcasting_correction for single output

import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Assuming `scaler`, `y_test`, and `predictions` are defined elsewhere in your code
# Inverse transform the predictions and y_test to get the actual values
y_test_actual = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], data.shape[1]-1))), axis=1))[:, 0]
y_pred_actual = scaler.inverse_transform(np.concatenate((decoded_sentence, np.zeros((predictions.shape[0], data.shape[1]-1))), axis=1))[:, 0]

# Step 2: Apply Nowcasting Correction using Extra Trees
def apply_nowcasting_correction(y_test_actual, y_pred_actual, percentiles=[0, 5, 95, 100]):
    corrections = []

    # Split data into percentiles
    for i in range(len(percentiles) - 1):
        lower_bound = np.percentile(y_pred_actual, percentiles[i])
        upper_bound = np.percentile(y_pred_actual, percentiles[i+1])

        # Select data within current percentile range
        mask = (y_pred_actual >= lower_bound) & (y_pred_actual < upper_bound)
        y_pred_segment = y_pred_actual[mask]
        y_test_segment = y_test_actual[mask]

        if len(y_pred_segment) > 1:
            # Calculate error (observed - predicted)
            error = y_test_segment - y_pred_segment

            # Perform Extra Trees regression
            reg = ExtraTreesRegressor(n_estimators=100, random_state=42)
            reg.fit(y_pred_segment.reshape(-1, 1), error)

            # Apply correction
            correction = y_pred_segment + reg.predict(y_pred_segment.reshape(-1, 1))
            corrections.append((mask, correction))

    # Combine all corrections
    y_corrected = np.copy(y_pred_actual)
    for mask, correction in corrections:
        y_corrected[mask] = correction

    return y_corrected

# Define the percentiles for grouping
percentiles = list(range(0, 95, 5)) + list(range(95, 101, 2))

# Apply correction
y_corrected = apply_nowcasting_correction(y_test_actual[1:1000], y_pred_actual, percentiles)

# Step 3: Evaluate the Performance
# Calculate RMSE and Correlation
rmse_before = np.sqrt(mean_squared_error(y_test_actual[1:1000], y_pred_actual))
corr_before = np.corrcoef(y_test_actual[1:1000], y_pred_actual)[0, 1]

rmse_after = np.sqrt(mean_squared_error(y_test_actual[1:1000], y_corrected))
corr_after = np.corrcoef(y_test_actual[1:1000], y_corrected)[0, 1]

print(f"RMSE before correction: {rmse_before}")
print(f"Correlation before correction: {corr_before}")
print(f"RMSE after correction: {rmse_after}")
print(f"Correlation after correction: {corr_after}")

# Step 4: Plotting the results
plt.figure(figsize=(14, 5))
plt.plot(y_test_actual[:100], label='Actual Precipitation')
plt.plot(y_pred_actual[:100], label='Predicted Precipitation (Before Correction)')
plt.plot(y_corrected[:100], label='Predicted Precipitation (After Correction)')
plt.legend()
plt.title('Precipitation Prediction')
plt.xlabel('Time Steps')
plt.ylabel('Precipitation')
plt.show()

# Plotly Visualization
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test_actual[1:30], mode='lines', name='Actual Precipitation', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=y_pred_actual[1:30], mode='lines', name='Predicted Precipitation (Before Correction)', line=dict(color='red')))
fig.add_trace(go.Scatter(y=y_corrected[1:30], mode='lines', name='Predicted Precipitation (After Correction)', line=dict(color='black')))
fig.update_layout(title='Extra Trees regression',
                  xaxis_title='Time',
                  yaxis_title='Precipitation',
                  legend_title='Legend')

fig.show()


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 31542 and the array at index 1 has size 5257